Author: Maxime Marin  
@: mff.marin@gmail.com

# Accessing IMOS data case studies: Walk-through and interactive session - Interactive

This notebook shows an interactive way of visualising the data selected in the "Start.ipynb" notebook.  
The goal of this interactive coding is to enable users without python knowledge to easily plot basic statistics of the dataset they chose for prelimnary analysis or use of figures.

***

## 1) Imports

Let's start with imports of libraries and data we selected previously.


In [8]:
import sys
import os
sys.path.append('/home/jovyan/intake-aodn')
import intake_aodn
from intake_aodn.analysis import time_average, lin_trend
from intake_aodn.plot import map_var
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs1
import cmocean
import xarray as xr

In [9]:
data = xr.open_dataset('Example_Data.nc')

Every notebook starts their own environments so we have to import the `import_aodn` again, along with other libraires used in this notebook.

`xr.open_dataset` loads the data we extracted in the previous notebook, so we do not have to load it again from the cloud.  
Now that we loaded the data, we have access to much more details, have a look at "Attributes"!

Most products "flag" the quality of their data to indicate where and when the data might be unreliable. Let's get rid of bad data:

In [10]:
data.quality_level.attrs

{'comment': 'These are the overall quality indicators and are used for all GHRSST SSTs. In this case they are a function of distance to cloud, satellite zenith angle, and day/night',
 'flag_meanings': 'no_data bad_data worst_quality low_quality acceptable_quality best_quality',
 'flag_values': array([0, 1, 2, 3, 4, 5]),
 'long_name': 'quality level of SST pixel',
 'valid_max': 5,
 'valid_min': 0,
 'coordinates': 'time lat lon'}

Using the `.attrs` gives us more details about the variable itself. We see that good and excellent quality are flagged 4 and 5, let's get rid of the rest:

In [11]:
data['sea_surface_temperature']-= 273.15 # Kelvin to Celsuis conversion
data['sea_surface_temperature'] = data['sea_surface_temperature'].where(data['quality_level'] >=4)# keep only good quality data

In [12]:
data

<xarray.Dataset>
Dimensions:                  (time: 8312, latitude: 100, longitude: 100)
Coordinates:
  * latitude                 (latitude) float32 -25.01 -25.03 ... -26.97 -26.99
  * longitude                (longitude) float32 112.5 112.5 ... 114.5 114.5
  * time                     (time) datetime64[ns] 1998-01-01T15:20:00 ... 20...
Data variables:
    sses_bias                (time, latitude, longitude) float32 ...
    l2p_flags                (time, latitude, longitude) float32 ...
    sst_dtime                (time, latitude, longitude) float64 ...
    sses_count               (time, latitude, longitude) float32 ...
    sea_surface_temperature  (time, latitude, longitude) float32 23.56 ... nan
    sses_standard_deviation  (time, latitude, longitude) float32 ...
    satellite_zenith_angle   (time, latitude, longitude) float32 ...
    quality_level            (time, latitude, longitude) float32 5.0 5.0 ... nan
    SST_at_depth             (time, latitude, longitude) float32 ...

***

## 2) Location TimeSeries

Our data is now ready for visualisation, let's run the cell below. This will output an interactive graph...Let's give it a go!

In [13]:
%matplotlib inline 
@interact
def plot_ts(longitude=(data.longitude.values.min(),
                       data.longitude.values.max(),
                       np.abs(np.diff(data.longitude).mean())),
            latitude=(data.latitude.values.min(),
                      data.latitude.values.max(),
                      np.abs(np.diff(data.latitude).mean())),
            time_step = ['month','year'],variable = list(data.keys())):

    ylab = variable
    freq = {'month':'M','year':'Y'}
    if time_step == 'day':
        test = data[variable]
    else:
        test = time_average(data,freq[time_step],var=ylab,ignore_inc = False)
      
    fig = plt.figure(figsize=(30,8))
    plt.subplot(1,2,1)
    coef,fit,hci,lci = lin_trend(test.sel(longitude=longitude,latitude=latitude),'time')
    fit = fit.assign(hci = hci['linear_fit'], lci = lci['linear_fit'])

    test.sel(longitude=longitude,latitude=latitude).plot() 
    fit['linear_fit'].plot(color='red') 
    xl = test.coords['time'].values
    plt.xlim(xl.min(),xl.max())
    
    plt.fill_between(lci['time'].values,lci['linear_fit'].values,hci['linear_fit'].values,alpha=0.5,color='grey')
    plt.gca().set_ylabel(ylab,fontsize = 20)
    plt.gca().set_xlabel(plt.gca().get_xlabel(),fontsize = 20)
    plt.gca().tick_params(labelsize = 15)
    plt.gca().set_title(time_step+'ly mean - lon:'+str(round(longitude,5))+ ' ; lat:'+str(round(latitude,5)),fontsize = 20);
    
    plt.subplot(1,4,3,projection=ccrs1.PlateCarree())
    time_mean = data[ylab].mean(dim='time')
    np.isnan(data[ylab]).sum(dim = 'time')
    ax,gl,axproj = map_var(time_mean,[data.longitude.min(),data.longitude.max()],[data.latitude.min(),data.latitude.max()],cmap = cmocean.cm.thermal,add_colorbar = True,title='Time-Mean',cbar_kwargs={'shrink':0.6})
    ax.set_title(ax.get_title(),fontsize = 20)
    gl.xlabel_style = {'size':12}
    gl.ylabel_style = {'size':12}
    plt.scatter(longitude,latitude,s=55,marker="o",edgecolor = 'black',color = 'white',zorder=3)
    fig1 = plt.gcf()
    plt.show()
    
    def save_fig(button):
        fig1.savefig("ts_location_lon {:.2f} _lat {:.2f}.png".format(longitude,latitude))
    save_but = widgets.Button(description='Save Figure')
    display(save_but)
    save_but.on_click(save_fig)
    
    return

interactive(children=(FloatSlider(value=112.51000213623047, description='longitude', max=114.48999786376953, m…

We have created an environment where users can freely select different locations and variables which will update the plot accordingly  

Note that the map on the left shows the time-mean state of the variable chosen

The time_step dropdown box allows the user to swtich from monthly to yearly means.

Finally, the user can save the figure by either right-clicking on it, or press the 'Save Figure' button. The latter will save the figure directly in the working directory. 

***

## 3) Interactive Map

We can dig further into basic visualisation by creating an interactive map which allows us to have a better feel of what the data looks like.  
Run the next cell and experiment!


In [14]:
# Interactive Map

import calendar
from intake_aodn.plot import create_cb
from time import strptime


day_list = np.arange(0,31,1)
month_list = [calendar.month_abbr[x] for x in np.arange(0,12,1)]
year_list = np.append(0,np.unique(data.time.dt.year))

print("Specifying '0' for Day/Month/Year will perform a monthly/yearly/time average")
#box_layout = Layout(display = 'flex',flex_flow = 'row')
@interact
def plot_ts(Day = day_list, Month = month_list, Year = year_list,variable = list(data.keys())):
    ylab = variable
    day_sel = np.arange(1,32,1) if Day == 0 else Day
    mn_sel = np.arange(1,13,1) if Month == '' else strptime(Month,'%b').tm_mon
    yr_sel = np.arange(data.time.dt.year.min(),data.time.dt.year.max()+1,1) if Year == 0 else Year
    
    date_str = str(Year) + '-' + str(Month) + '-' + str(Day)
    
    title_yr = f"{Year:04}"
    title_mn =  Month
    title_day = f"{Day:02}"
    if Year == 0:
        title_yr = 'All'
    if Month == '':
        title_mn = 'All'
    if Day == 0:
        title_day = 'All'
    if (Year == 0) & (Month == 0) & (Day == 0):
        title = 'Time-Mean'
    else:
        title = 'Year: {}   Month: {}   Day: {}'.format(title_yr,title_mn,title_day)

    da = data[ylab][(data.time.dt.year.isin(yr_sel)) & (data.time.dt.month.isin(mn_sel)) & data.time.dt.day.isin(day_sel)].mean(dim = 'time')
    fig1 = plt.figure(figsize=(20,12))
    ax,gl,axpl = map_var(da,[data.longitude.min(),data.longitude.max()],[data.latitude.min(),data.latitude.max()],cmap = cmocean.cm.thermal,add_colorbar = False,title=title)
    cb = create_cb(fig1,ax,axpl,variable,size = "4%", pad = 0.5,labelpad = 20,fontsize=20)
    cb.ax.tick_params(labelsize = 15)
    ax.set_title(ax.get_title(),fontsize = 25)
    gl.xlabel_style = {'size':15}
    gl.ylabel_style = {'size':15}
    
    def save_fig(button):
        fig1.savefig("map_yr{}_mn{}_dy{}.png".format(title_yr,title_mn,title_day))
    save_but = widgets.Button(description='Save Figure')
    display(save_but)
    save_but.on_click(save_fig)


Specifying '0' for Day/Month/Year will perform a monthly/yearly/time average


interactive(children=(Dropdown(description='Day', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

Et voila, we can now map the state of our selected region and chose what day we want to look at.  

Notice that you can set values of the day month and year to zero (no value for month)? This will indicate that you do not care about that time attribute: day 0 will perform a monthly average, month and day 0 will perform a yearly average... and so forth.